# Classifier testing

In [1]:
import lib.database_module as db
import lib.encoding_module as en
import lib.download_mine as down
from sklearn.externals import joblib
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics.pairwise import cosine_similarity
import tqdm
import sklearn.discriminant_analysis as da
import sklearn.cluster as cluster
import sklearn.neighbors as neighbors
import sklearn.neural_network as nn
from sklearn.model_selection import GridSearchCV

In [2]:
vectors= db.fetch_all_category_vectors('local')

In [3]:
y = []
X = []

for v in vectors:
    v[1].insert(0,v[0])
    x = np.array(v[1])
    X.append(x)
vector_df = pd.DataFrame(X)
X_train, X_test, y_train, y_test = train_test_split(vector_df.iloc[:,1:],vector_df[0])

In [4]:
vectorizer = joblib.load('data/vectorizer.pkl')

Subcategories_dict = {'Sandwiches':'American sandwiches',
                'desserts':'cookies',
                'chemistry':'Industrial gases',
                'physics':'physicists',
                'sports cars': 'muscle cars',
                'psychology':'Popular psychology',
                'Arcade games':'Cancelled arcade games',
                'machine learning':'Classification algorithms',
                'cat breeds':'Natural cat breeds',
                'Earth sciences':'Geochemistry',
                'Breads':'Bread dishes',
                'Automotive technologies':'Vehicle dynamics',
                'belief':'Ignorance',
                'hygiene':'Ritual purification',
                'sports terminology':'martial arts terminology',
                'shoes':'Sneaker culture',
                'influenza':'Influenza researchers',
                'Children':'Sons of Odin',
                'physical quantities':'Density',
                'Physical exercise':'hydrotherapy',
                'health care':'medical robotics'
                }

Subcategory_number_dict = {}

for k,v in tqdm.tqdm_notebook(Subcategories_dict.items()):
    parent = down.wikipedia_get_page(k, category=True)
    child =  down.wikipedia_get_page(v, category=True)
    Subcategory_number_dict[v] ={'parent_name':k,
                                 'parent_id':int(parent.keys()[0]) , 
                                 'category_id': int(child.keys()[0]),
                                 'pages' :{p['pageid']: down.get_text(p['pageid']) 
                                           for p in down.wikipedia_get_pages_for_category(v) 
                                           if p['ns'] == 0}}
X_validation = []
y_validation = []
for k, v in Subcategory_number_dict.items():
    for key,text in v['pages'].items():
        y_validation.append(v['parent_id'])
        X_validation.append(en.clean_page_text(text))
X_validation = vectorizer.transform(X_validation)


{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}



## k Nearest Neighbors

In [ ]:
knn = neighbors.KNeighborsClassifier()
knn.fit(X_train, y_train)

In [ ]:
gs_params = {'algorithm':['auto', 'ball_tree' ],
             'p':[3], 'n_neighbors' : [3,5,7],
             'weights': ['uniform' ,'distance']}
gs = GridSearchCV(knn,param_grid=gs_params, n_jobs =3, verbose =2)

In [ ]:
gs.fit(X_train,y_train)
gs.score(X_train,y_train)
joblib.dump(gs,"knn_gs")

In [ ]:
print "Training Set Accuracy: {:.1%}".format(knn.score(X_train,y_train))
print "Testing Set Accuracy: {:.1%}".format(gs.score(X_test,y_test))
print "Validation Set Accuracy: {:.1%}".format(gs.score(X_validation,y_validation))

In [ ]:
gs.best_estimator_

In [ ]:
knn_best = neighbors.KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=2, n_neighbors=5, p=3,
           weights='distance')
knn_best.fit(X_train, y_train)
print "Training Set Accuracy: {:.1%}".format(knn_best.score(X_train,y_train))
print "Testing Set Accuracy: {:.1%}".format(knn_best.score(X_test,y_test))
print "Validation Set Accuracy: {:.1%}".format(knn_best.score(X_validation,y_validation))

In [ ]:
gs.best_estimator_

In [ ]:
import sklearn.neural_network as nn
mlpc = nn.MLPClassifier(verbose =1,
                        max_iter=2000, 
                        activation = 'logistic',
                        solver='adam',
                        hidden_layer_sizes=(100,) ,
                        tol=.00001)
                    
mlpc.fit(X_train, y_train)

In [ ]:

pred= mlpc.predict(X_test)
mlpc_results = zip(pred,y_test)
results = []
for m in mlpc_results:
    
    if m[0] == m[1]:
        results.append(int(1))
    else:
        results.append(int(0))
accuracy = float(sum(results))/len(results)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [ ]:
da_class = da.LinearDiscriminantAnalysis()
da_class.fit_transform(X_train,y_train)

In [ ]:
da_class.predict(X_test)

In [ ]:
pred= da_class.predict(X_test)
pred_results = zip(pred,y_test)
results = []
for k in pred_results:
    if k[0] == k[1]:
        results.append(int(1))
    else:
        results.append(int(0))
accuracy = float(sum(results))/len(results)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [7]:
import sklearn.ensemble as ens

In [9]:
%%timeit tree = ens.RandomForestClassifier(n_estimators=30,n_jobs=2, verbose=2)
tree.fit(X_train,y_train)

building tree 1 of 30building tree 2 of 30

building tree 3 of 30
building tree 4 of 30
building tree 5 of 30
building tree 6 of 30
building tree 7 of 30
building tree 8 of 30
building tree 9 of 30
building tree 10 of 30
building tree 11 of 30
building tree 12 of 30
building tree 13 of 30
building tree 14 of 30
building tree 15 of 30
building tree 16 of 30
building tree 17 of 30
building tree 18 of 30
building tree 19 of 30
building tree 20 of 30
building tree 21 of 30
building tree 22 of 30
building tree 23 of 30
building tree 24 of 30
building tree 25 of 30
building tree 26 of 30
building tree 27 of 30
building tree 28 of 30
building tree 29 of 30
building tree 30 of 30


[Parallel(n_jobs=2)]: Done  30 out of  30 | elapsed:    1.3s finished


building tree 2 of 30building tree 1 of 30

building tree 3 of 30
building tree 4 of 30
building tree 5 of 30
building tree 6 of 30
building tree 7 of 30
building tree 8 of 30
building tree 9 of 30
building tree 10 of 30
building tree 11 of 30
building tree 12 of 30
building tree 13 of 30
building tree 14 of 30
building tree 15 of 30
building tree 16 of 30
building tree 17 of 30
building tree 18 of 30
building tree 19 of 30
building tree 20 of 30
building tree 21 of 30
building tree 22 of 30
building tree 23 of 30
building tree 24 of 30
building tree 25 of 30
building tree 26 of 30
building tree 27 of 30
building tree 28 of 30
building tree 29 of 30
building tree 30 of 30


[Parallel(n_jobs=2)]: Done  30 out of  30 | elapsed:    1.8s finished


building tree 1 of 30
building tree 2 of 30
building tree 3 of 30 
building tree 4 of 30
building tree 5 of 30
 building tree 6 of 30
building tree 7 of 30
building tree 8 of 30
building tree 9 of 30
building tree 10 of 30
building tree 11 of 30
building tree 12 of 30
building tree 13 of 30
building tree 14 of 30
building tree 15 of 30
building tree 16 of 30
building tree 17 of 30
building tree 18 of 30
building tree 19 of 30
building tree 20 of 30
building tree 21 of 30
building tree 22 of 30
building tree 23 of 30
building tree 24 of 30
building tree 25 of 30
building tree 26 of 30
building tree 27 of 30
building tree 28 of 30
building tree 29 of 30
building tree 30 of 30


[Parallel(n_jobs=2)]: Done  30 out of  30 | elapsed:    1.3s finished


building tree 1 of 30
building tree 2 of 30
building tree 3 of 30
building tree 4 of 30
building tree 5 of 30
building tree 6 of 30
building tree 7 of 30
building tree 8 of 30
building tree 9 of 30
building tree 10 of 30
building tree 11 of 30
building tree 12 of 30
building tree 13 of 30
building tree 14 of 30
building tree 15 of 30
building tree 16 of 30
building tree 17 of 30
building tree 18 of 30
building tree 19 of 30
building tree 20 of 30
building tree 21 of 30
building tree 22 of 30
building tree 23 of 30
building tree 24 of 30
building tree 25 of 30
building tree 26 of 30
building tree 27 of 30
building tree 28 of 30
building tree 29 of 30
building tree 30 of 30
1 loop, best of 3: 1.61 s per loop


[Parallel(n_jobs=2)]: Done  30 out of  30 | elapsed:    1.8s finished


In [ ]:
print "Training Set Accuracy: {:.1%}".format(tree.score(X_train,y_train))
print "Testing Set Accuracy: {:.1%}".format(tree.score(X_test,y_test))

In [10]:
%%timeit 
print "Validation Set Accuracy: {:.1%}".format(tree.score(X_validation,y_validation))

[Parallel(n_jobs=2)]: Done  30 out of  30 | elapsed:    0.0s finished
[Parallel(n_jobs=2)]: Done  30 out of  30 | elapsed:    0.0s finished


Validation Set Accuracy: 67.3%
Validation Set Accuracy: 67.3%


[Parallel(n_jobs=2)]: Done  30 out of  30 | elapsed:    0.0s finished
[Parallel(n_jobs=2)]: Done  30 out of  30 | elapsed:    0.0s finished


Validation Set Accuracy: 67.3%
Validation Set Accuracy: 67.3%


[Parallel(n_jobs=2)]: Done  30 out of  30 | elapsed:    0.0s finished
[Parallel(n_jobs=2)]: Done  30 out of  30 | elapsed:    0.0s finished


...

[Parallel(n_jobs=2)]: Done  30 out of  30 | elapsed:    0.0s finished
[Parallel(n_jobs=2)]: Done  30 out of  30 | elapsed:    0.1s finished


Validation Set Accuracy: 67.3%
Validation Set Accuracy: 67.3%


[Parallel(n_jobs=2)]: Done  30 out of  30 | elapsed:    0.0s finished
[Parallel(n_jobs=2)]: Done  30 out of  30 | elapsed:    0.0s finished


Validation Set Accuracy: 67.3%
Validation Set Accuracy: 67.3%
Validation Set Accuracy: 67.3%
10 loops, best of 3: 120 ms per loop


[Parallel(n_jobs=2)]: Done  30 out of  30 | elapsed:    0.1s finished
[Parallel(n_jobs=2)]: Done  30 out of  30 | elapsed:    0.0s finished


In [5]:
from sklearn.linear_model import SGDClassifier

In [11]:
.12/len(X_validation
       )

0.00011964107676969092

In [6]:
sgd = SGDClassifier(n_jobs=2, verbose=2, penalty='l2' )
sgd.fit(X_train,y_train)
print "Training Set Accuracy: {:.1%}".format(sgd.score(X_train,y_train))
print "Testing Set Accuracy: {:.1%}".format(sgd.score(X_test,y_test))
print "Validation Set Accuracy: {:.1%}".format(sgd.score(X_validation,y_validation))

-- Epoch 1
-- Epoch 1
Norm: 10.19, NNZs: 500, Bias: 0.940001, T: 3555, Avg. loss: 0.212448
Total training time: 0.01 seconds.
Norm: 6.19, NNZs: 500, Bias: -1.093195, T: 3555, Avg. loss: 0.144225-- Epoch 2

Total training time: 0.01 seconds.
-- Epoch 2
Norm: 11.33, NNZs: 500, Bias: -1.266264, T: 7110, Avg. loss: 0.161398Norm: 6.58, NNZs: 500, Bias: -1.161565, T: 7110, Avg. loss: 0.114501

Total training time: 0.01 seconds.Total training time: 0.01 seconds.

-- Epoch 3-- Epoch 3

Norm: 11.86, NNZs: 500, Bias: -1.262905, T: 10665, Avg. loss: 0.137788
Total training time: 0.02 seconds.
-- Epoch 4
Norm: 6.92, NNZs: 500, Bias: -1.162056, T: 10665, Avg. loss: 0.099887
Total training time: 0.02 seconds.
-- Epoch 4
Norm: 12.14, NNZs: 500, Bias: -1.269053, T: 14220, Avg. loss: 0.123485
Total training time: 0.02 seconds.
-- Epoch 5
Norm: 7.13, NNZs: 500, Bias: -1.162145, T: 14220, Avg. loss: 0.090836
Total training time: 0.02 seconds.
-- Epoch 5
Norm: 12.30, NNZs: 500, Bias: -1.270803, T: 17775, 

[Parallel(n_jobs=2)]: Done  23 out of  23 | elapsed:    0.3s finished


In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
def scoring_mlp(estimator, X_test, y_test):
    

In [ ]:
parameters = {'hidden_layer_sizes':((10,),(10,100),(100,10),(100,)),
              'solver':('sgd', 'adam'), 
              'activation':('logistic', 'tanh', 'relu')}
estimator = nn.MLPClassifier(verbose =1,
                        max_iter=2000,  
                        tol=.0001)

In [ ]:
gs = GridSearchCV(estimator, parameters, cv =2)

In [ ]:
gs.fit(X_train, y_train)

In [ ]:
gs.best_params_
